<a href="https://colab.research.google.com/github/keymemory/Machine_Learning_Second/blob/main/ML2_day5_English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:

from sklearn import svm, metrics
import glob, os.path, re, json

# 텍스트를 읽어 들이고 출현 빈도 조사하기 --- (※1)
def check_freq(fname):
    name = os.path.basename(fname)
    # 파일 이름 앞의 2 문자가 언어 코드이므로 이를 기반으로 언어 ID를 추출한다.
    lang = re.match(r'^[a-z]{2,}', name).group()
    with open(fname, "r", encoding="utf-8") as f:
        text = f.read()
    text = text.lower() # 소문자 변환
    # 숫자 세기 변수(cnt) 초기화하기
    cnt = [0 for n in range(0, 26)]
    code_a = ord("a")
    code_z = ord("z")

    # 알파벳 출현 횟수 구하기 --- (※2)
    # 즉, a~z까지의 알파벳이 ac 번씩 출현하는지 조사한다.
    for ch in text:
        n = ord(ch)
        if code_a <= n <= code_z: # a~z 사이에 있을 때
            cnt[n - code_a] += 1

    # 정규화하기 --- (※3)
    total = sum(cnt)
    # 알파벳의 출현 횟수를 글자수의 합계로 나누어 출현 횟수를 출현 빈도로 변환한다.
    # 출현 횟수를 출현 빈도로 변환하는 이유는 각 텍스트 파일에 있는 문자 수가 다르기 
    # 때문에 출현 횟수를 비교하면 동일한 규칙이 적용되지 않기 때문이다.
    # 따라서 모든 규칙을 통일하는 것인데, 이를 정규화라 한다.
    # 정규화란, 데이터를 일정한 규칙을 기반으로 변형해서 쉽게 사용할 수 있게 한다.
    freq = list(map(lambda n: n / total, cnt))
    return (freq, lang)
    
# 각 파일 처리하기
# 출현 횟수를 조사할 때는 글자를 하나씩 꺼내서 문자 코드로 고친 뒤 해당 문자 코드를 
# 기반으로 배열의 특정 위치에 있는 숫자를 하나씩 증가시키는 방법을 사용한다.
def load_files(path):
    freqs = []
    labels = []
    file_list = glob.glob(path)
    for fname in file_list:
        r = check_freq(fname)
        freqs.append(r[0])
        labels.append(r[1])
    return {"freqs":freqs, "labels":labels}
data = load_files("./lang/train/*.txt")
test = load_files("./lang/test/*.txt")

# 이후를 대비해서 JSON으로 결과 저장하기
with open("./lang/freq.json", "w", encoding="utf-8") as fp:
    json.dump([data, test], fp)

# 학습하기 --- (※4)
clf = svm.SVC()
clf.fit(data["freqs"], data["labels"])

# 예측하기 --- (※5)
predict = clf.predict(test["freqs"])

# 결과 테스트하기 --- (※6)
ac_score = metrics.accuracy_score(test["labels"], predict)
cl_report = metrics.classification_report(test["labels"], predict)
print("정답률 =", ac_score)
print("리포트 =")
print(cl_report)

FileNotFoundError: ignored